# Data import

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [6]:
pd.set_option('display.max_columns', None) # Show all the columns
df = pd.read_csv('combined_data.csv')
df = df.drop(columns='Unnamed: 0')
df.head()

,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways,blocked,startRinkSide,type,date_time_GMT,home_rink_side_start,venue,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz,timeOnIce,evenTimeOnIce,shortHandedTimeOnIce,powerPlayTimeOnIce,goalie_replacement
0,2016020045,4,away,False,REG,Dave Hakstol,4.0,27.0,30.0,6.0,4.0,2.0,50.9,12.0,9.0,11.0,left,R,2016-10-19T00:30:00Z,right,United Center,America/Chicago,-5,CDT,990.833333,841.388889,55.555556,93.888889,Yes
1,2016020045,16,home,True,REG,Joel Quenneville,7.0,28.0,20.0,8.0,3.0,2.0,49.1,16.0,8.0,9.0,left,R,2016-10-19T00:30:00Z,right,United Center,America/Chicago,-5,CDT,981.333333,836.777778,75.111111,69.444444,No
2,2017020812,24,away,True,OT,Randy Carlyle,4.0,34.0,16.0,6.0,3.0,1.0,43.8,7.0,4.0,14.0,right,R,2018-02-07T00:00:00Z,left,KeyBank Center,America/New_York,-4,EDT,1002.222222,879.611111,28.444444,94.166667,No
3,2017020812,7,home,False,OT,Phil Housley,3.0,33.0,17.0,8.0,2.0,1.0,56.2,5.0,6.0,14.0,right,R,2018-02-07T00:00:00Z,left,KeyBank Center,America/New_York,-4,EDT,999.222222,888.333333,75.333333,35.555556,No
4,2015020314,21,away,True,REG,Patrick Roy,4.0,29.0,17.0,9.0,3.0,1.0,45.7,13.0,5.0,20.0,left,R,2015-11-24T01:00:00Z,right,MTS Centre,America/Winnipeg,-5,CDT,986.666667,844.722222,53.333333,88.611111,No


# CausalML

## Import packages

In [1]:
import scipy.stats
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from causalml.inference.meta import BaseSRegressor, BaseTRegressor, BaseXRegressor, BaseRRegressor
from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier


from causalml.inference.meta import LRSRegressor
from causalml.inference.meta import XGBTRegressor
from causalml.inference.meta import BaseSRegressor

import shap
import matplotlib.pyplot as plt

import time
from causalml.metrics import *
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2
%matplotlib inline

sklearn.tree._tree.TreeBuilder size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject


In [2]:
import causalml
print(causalml.__version__)

0.12.0


In [3]:
# check scikit-learn version
import sklearn
print(sklearn.__version__)

0.24.2


## Identify treatment, x, y variables

In [ ]:
# Treatment feature


In [ ]:
# Target variable (y)


In [ ]:
# Control variables (X)


In [ ]:
# w_multi
w_multi = np.array(['treatment_A' if x==1 else 'control' for x in treatment])

In [ ]:
# Propensity calculation


In [ ]:
# Feature name

## Modelling

### LRSRegressor with SLearner, "auto" method

In [ ]:
# SLearner & Auto
# Ready-to-use S-Learner using LinearRegression
learner_s = LRSRegressor()
cate_s = learner_s.estimate_ate(X=X, treatment=treatment, y=y_rev)
print(cate_s)
print('ATE estimate: {:.03f}'.format(cate_s[0][0]))
print('ATE lower bound: {:.03f}'.format(cate_s[1][0]))
print('ATE upper bound: {:.03f}'.format(cate_s[2][0]))

In [ ]:
slearner_tau = learner_s.fit_predict(X, treatment, y)

In [ ]:
learner_s.get_importance(X=X, 
                         tau=slearner_tau,
                         normalize=True, 
                         method='auto',
                         features = feature_names)

In [ ]:
learner_s.plot_importance(X=X, 
                         tau=slearner_tau, 
                         normalize=True, 
                         method='auto', 
                         features=feature_names)

### LRSRegressor with SLearner, "permutation" method

In [ ]:
slearner.get_importance(X=X, 
                        tau=slearner_tau, 
                        method='permutation', 
                        features=feature_names)

In [ ]:
learner_s.plot_importance(X=X, 
                         tau=slearner_tau, 
                         normalize=True, 
                         method='permutation', 
                         features=feature_names)

### LRSRegressor with SLearner, sklearn.inspection.permutation_importance

In [ ]:
start_time = time.time()

X_train, X_test, y_train, y_test = train_test_split(X, slearner_tau, test_size=0.3, random_state=42)
model_tau_fit = model_tau.fit(X_train, y_train)

perm_imp_test = permutation_importance(
    estimator=model_tau_fit, 
    X=X_test, 
    y=y_test, 
    random_state=42).importances_mean
pd.Series(perm_imp_test, feature_names).sort_values(ascending=False)

print("Elapsed time: %s seconds" % (time.time() - start_time))

In [ ]:
pd.Series(perm_imp_test, feature_names).sort_values(ascending=False)

In [ ]:
pd.Series(perm_imp_test, feature_names).sort_values().plot(kind='barh', figsize=(12, 8))
plt.title('Test Set Permutation Importances')

In [ ]:
perm_imp_train = permutation_importance(
    estimator=model_tau_fit, 
    X=X_train, 
    y=y_train, 
    random_state=42).importances_mean
pd.Series(perm_imp_train, feature_names).sort_values(ascending=False)

In [ ]:
pd.Series(perm_imp_train, feature_names).sort_values().plot(kind='barh', figsize=(12, 8))
plt.title('Training Set Permutation Importances')

### LRSRegressor with SLearner, Shapley Values

In [ ]:
shap_slearner = slearner.get_shap_values(X=X, tau=slearner_tau)
shap_slearner

In [ ]:
np.mean(np.abs(shap_slearner['treatment_A']),axis=0)

In [ ]:
# Plot shap values without specifying shap_dict
slearner.plot_shap_values(X=X, tau=slearner_tau, features=feature_names)

In [ ]:
# Plot shap values WITH specifying shap_dict
slearner.plot_shap_values(X=X, shap_dict=shap_slearner)

In [ ]:
# interaction_idx set to None (no color coding for interaction effects)
slearner.plot_shap_dependence(treatment_group='treatment_A',
                              feature_idx=1,
                              X=X,
                              tau=slearner_tau,
                              interaction_idx=None,
                              shap_dict=shap_slearner)

In [ ]:
# interaction_idx set to 'auto' (searches for feature with greatest approximate interaction)
# specify feature names
slearner.plot_shap_dependence(treatment_group='treatment_A',
                              feature_idx='tiger',
                              X=X,
                              tau=slearner_tau,
                              interaction_idx='auto',
                              shap_dict=shap_slearner,
                              features=feature_names)

In [ ]:
# interaction_idx set to specific index
slearner.plot_shap_dependence(treatment_group='treatment_A',
                              feature_idx=1,
                              X=X,
                              tau=slearner_tau,
                              interaction_idx=10,
                              shap_dict=shap_slearner, 
                              features=feature_names)

### LRSRegressor with TLearner, "auto" method

In [ ]:
learner_t = LRSRegressor()
cate_t = learner_t.estimate_ate(X=X, treatment=treatment, y=y_rev)
print(cate_t)
print('ATE estimate: {:.03f}'.format(cate_t[0][0]))
print('ATE lower bound: {:.03f}'.format(cate_t[1][0]))
print('ATE upper bound: {:.03f}'.format(cate_t[2][0]))

tlearner_tau = learner_t.fit_predict(X, treatment, y)

In [ ]:
learner_t.get_importance(X=X, 
                         tau=tlearner_tau,
                         normalize=True, 
                         method='auto',
                         features = feature_names)

In [ ]:
learner_t.plot_importance(X=X, 
                         tau=tlearner_tau, 
                         normalize=True, 
                         method='auto', 
                         features=feature_names)

### XGBTRegressor with TLearner, "auto" method

In [ ]:
# T Learner
learner_t = BaseTRegressor(learner=XGBRegressor())
ate_t = learner_t.estimate_ate(X=X, treatment=treatment, y=y_rev)
cate_t = learner_t.fit_predict(X=X, treatment=treatment, y=y_rev)
print('Using the ready-to-use XGBTRegressor class')
print(ate_t)

In [ ]:
learner_t.get_importance(X=X, 
                        tau=cate_t,
                        normalize=True, 
                        method='auto', 
                        features=feature_names)

In [ ]:
learner_t.plot_importance(X=X, 
                         tau=cate_t, 
                         normalize=True, 
                         method='auto', 
                         features=feature_names)

### XGBTRegressor with XLearner, without Propensity score input

In [ ]:
# X Learner without propensity score input
learner_x_no_p = BaseXRegressor(learner=XGBRegressor())
ate_x = learner_x_no_p.estimate_ate(X=X, treatment=treatment, y=y_rev)
print('Using the BaseXRegressor class and using XGB without propensity score input:')
print(ate_x)
cate_x_no_p = learner_x_no_p.fit_predict(X=X, treatment=treatment, y=y_rev)

In [ ]:
learner_x_no_p.get_importance(X=X, 
                        tau=cate_x_no_p,
                        normalize=True, 
                        method='auto', 
                        features=feature_names)

In [ ]:
learner_x_no_p.plot_importance(X=X, 
                         tau=cate_x_no_p, 
                         normalize=True, 
                         method='auto', 
                         features=feature_names)

### XGBTRegressor with RLearner without propensity score input

In [ ]:
# R Learner without propensity score input
learner_r_no_p = BaseRRegressor(learner=XGBRegressor())
ate_x = learner_r_no_p.estimate_ate(X=X, treatment=treatment, y=y_rev)
print('Using the BaseXRegressor class and using XGB without propensity score input:')
print(ate_x)
cate_r_no_p = learner_r_no_p.fit_predict(X=X, treatment=treatment, y=y_rev)

In [ ]:
learner_r_no_p.get_importance(X=X, 
                        tau=cate_r_no_p,
                        normalize=True, 
                        method='auto', 
                        features=feature_names)

In [ ]:
learner_r_no_p.plot_importance(X=X, 
                         tau=cate_r_no_p, 
                         normalize=True, 
                         method='auto', 
                         features=feature_names)

### XGBTRegressor with X Learner with propensity score input

In [ ]:
# X Learner with propensity score input
learner_x = BaseXRegressor(learner=XGBRegressor())
cate_x = learner_x.fit_predict(X=X, treatment=w_multi, y=y,p=e)

### XGBTRegressor with R Learner with propensity score input 

In [ ]:
# R Learner with propensity score input 
learner_r = BaseRRegressor(learner=XGBRegressor())
cate_r = learner_r.fit_predict(X=X, treatment=w_multi, y=y,p=e)

### A distribution plot of different learners of XGBTRegressor

In [ ]:
alpha=0.2
bins=30
plt.figure(figsize=(12,8))
plt.hist(cate_t, alpha=alpha, bins=bins, label='T Learner')
plt.hist(cate_x, alpha=alpha, bins=bins, label='X Learner')
plt.hist(cate_x_no_p, alpha=alpha, bins=bins, label='X Learner (no propensity score)')
plt.hist(cate_r, alpha=alpha, bins=bins, label='R Learner')
plt.hist(cate_r_no_p, alpha=alpha, bins=bins, label='R Learner (no propensity score)')
# Consider having this
plt.vlines(cate_s[0], 0, plt.axes().get_ylim()[1], label='S Learner',
           linestyles='dotted', colors='green', linewidth=2)
# Or this
plt.axvline(x=-0.130 ,color="black", linestyle="--", label = "S Learner", linewidth=2)

plt.title('Distribution of CATE Predictions by Meta Learner')
plt.xlabel('Individual Treatment Effect (ITE/CATE)')
plt.ylabel('# of Samples')
_=plt.legend()

### Uplift Tree/Forest

In [ ]:
# UpliftTreeClassifier
from causalml.dataset import make_uplift_classification

df, x_names = make_uplift_classification()

In [ ]:
uplift_tree = UpliftTreeClassifier(control_name='control')

uplift_tree.fit(X=df[x_names].values,
                treatment=df['treatment_group_key'].values,
                y=df['conversion'].values)

In [ ]:
pd.Series(uplift_tree.feature_importances_, index=x_names).sort_values().plot(kind='barh', figsize=(12,8))

In [ ]:
# UpliftRandomForestClassifier
uplift_rf = UpliftRandomForestClassifier(control_name='control')

uplift_rf.fit(X=df[x_names].values,
              treatment=df['treatment_group_key'].values,
              y=df['conversion'].values)

In [ ]:
pd.Series(uplift_rf.feature_importances_, index=x_names).sort_values().plot(kind='barh', figsize=(12,8))